In [1]:
# 匯入函數

import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import LinearRegression
import progressbar as bar

In [2]:
# 資料篩選

def Filter_data(path):

    column = ['time','longitude','latitude','scale','depth']

    CWBdata = pd.read_csv(path, encoding='big5')
    Sortdata = pd.DataFrame(np.array(CWBdata[:len(CWBdata)].iloc[::-1]), columns=column)

    frequencytotal = np.unique(Sortdata['scale'], return_counts=True)
    frequencydict = dict(zip(frequencytotal[1], frequencytotal[0]))

    filterscale = Sortdata['scale'] >= frequencydict.get(max(frequencytotal[1]))
    filterdata = pd.DataFrame(np.array(Sortdata[filterscale]), columns=column).dropna(axis = 0)
    
    return filterdata

In [3]:
data = Filter_data('D:/Dataset/Earthquake/Original/CWB_593634_19990101_20201231.csv')

In [4]:
# 時間拆分

def Data_time_Split(data):
    
    data['time'] = pd.to_datetime(data['time'])
    data['year'] = pd.to_datetime(data['time']).dt.year
    data['month'] = pd.to_datetime(data['time']).dt.month
    data['day'] = pd.to_datetime(data['time']).dt.day
    
    data = data[['time','year','month','day','longitude','latitude','scale','depth']]
    
    return data

In [5]:
data = Data_time_Split(data)

In [6]:
# 規模概率計算

def A1516(Lower,Upper):
    
    A1516array,y = np.empty(len(data)),0
    total = data.time.iloc[-1]-data.time.iloc[0]
    mean = total.total_seconds()/len(data[(data.scale>=Lower)&(data.scale<Upper)])

    for x in np.arange(len(data)):

        if (data.scale[x]>=Lower)&(data.scale[x]<Upper):

            y=x

        A1516 = (data.time[x]-data.time[y]).total_seconds()/mean
        A1516array[x] = A1516
        
    return A1516array

In [7]:
# 每次機率

def Repeat_operation_Each(i):

    Probability = np.empty(shape=(int((max(data.scale)-min(data.scale))*(1/i)+1),len(data)))

    for x in bar.progressbar(np.arange(int((max(data.scale)-min(data.scale))*(1/i)+1))):
        
        if len(data[(data.scale>=min(data.scale)+i*x)&(data.scale<min(data.scale)+i*(x+1))])!=0:

            Probability[x] = A1516(min(data.scale)+i*x,min(data.scale)+i*(x+1))
            
        else:
            
            Probability[x] = 0
            
    Probabilitys = pd.DataFrame(Probability.T,columns=pd.DataFrame(Probability.T).columns*101+1516)
    Probabilitys.index = np.arange(len(Probabilitys))
        
    return Probabilitys[Probabilitys.columns[(Probabilitys==0).all()==False]]

In [8]:
Probabilityeach = Repeat_operation_Each(0.1) # 控制間隔

100% (59 of 59) |########################| Elapsed Time: 0:15:37 Time:  0:15:37


In [9]:
# 每天機率

def Repeat_operation_Day(startyear,endyear,startmonth,endmonth):
    
    Probabilityeach['year'] = data['year']
    Probabilityeach['month'] = data['month']
    Probabilityeach['day'] = data['day']

    years = np.arange(startyear,endyear)
    months = np.arange(startmonth,endmonth)
    
    P = pd.DataFrame()
        
    for year in bar.progressbar(years):
        
        daylist = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

        if (year%4==0 and year%100!=0) or (year%400==0) : 
            
            daylist = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        for month in months:
            
            days = np.arange(1,daylist[month-1]+1)

            for day in days:
                
                P = pd.concat([P,np.mean(Probabilityeach[(Probabilityeach['year']==year)&(Probabilityeach['month']==month)&(Probabilityeach['day']==day)])],axis=1)
                
    Probability = P.T
    Probability.index = np.arange(len(Probability))
    
    return Probability

In [10]:
Probabilityday = Repeat_operation_Day(1999,2021,1,13)

100% (22 of 22) |########################| Elapsed Time: 0:06:36 Time:  0:06:36


In [51]:
# 歷史古騰堡參數

def Gutenberg_Richter_total(scale):
    
    intercept,coef  = [],[]
    
    frequencytotal = np.unique(scale, return_counts=True)
    table = pd.DataFrame(frequencytotal, index=['scale', 'frequency']).T
    table['cumsum'] = table['frequency'].iloc[::-1].cumsum()
    table['log'] = np.log10(table['cumsum'])
    x = np.array([table['scale']]).T
    y = np.array([table['log']]).T
    LR = LinearRegression().fit(x, y)
    
    intercept.append(LR.intercept_[0])
    coef.append(-LR.coef_[0][0])
    
    return intercept,coef,table

In [52]:
GRtotal = Gutenberg_Richter_total(data['scale'])

In [77]:
pd.concat([GRtotal[2].iloc[:,0:1],GRtotal[2].iloc[:,2:3]],axis=1).to_excel('C:/Users/btea4/OneDrive/桌面/GRtotal2.xlsx',index=False)

In [13]:
# 區間古騰堡參數

def Gutenberg_Richter_interval(startyear,endyear,startmonth,endmonth):
    
    intercept,coef,GRloss,scalemaxexpected  = [],[],[],[]
    
    years = np.arange(startyear,endyear)
    months = np.arange(startmonth,endmonth)
        
    for year in years:
        
        daylist = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        if (year%4==0 and year%100!=0) or (year%400==0) : 
            
            daylist = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        for month in months:
            
            days = np.arange(1,daylist[month-1]+1)

            for day in days:
            
                scale = data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['scale']
                
                frequencytemporary = np.unique(scale, return_counts=True)
                table = pd.DataFrame(frequencytemporary, index=['scale', 'frequency']).T
                table['cumsum'] = table['frequency'].iloc[::-1].cumsum()
                table['log'] = np.log10(table['cumsum'])
                x = np.array([table['scale']]).T
                y = np.array([table['log']]).T
                LR = LinearRegression().fit(x, y)

                loss = sum((GRtotal[0]-GRtotal[1]*scale)-(LR.intercept_[0]-LR.coef_[0][0]*scale))/len(scale)
                
                expected = LR.intercept_[0]/-LR.coef_[0][0]

                intercept.append(LR.intercept_[0])
                coef.append(-LR.coef_[0][0])
                GRloss.append(loss)
                scalemaxexpected.append(expected)

    return intercept,coef,GRloss,scalemaxexpected

In [14]:
GRinterval = Gutenberg_Richter_interval(1999,2021,1,13)

In [15]:
# 區間平均規模

def scale_mean_interval(startyear,endyear,startmonth,endmonth):
    
    scalemean = []
    
    years = np.arange(startyear,endyear)
    months = np.arange(startmonth,endmonth)
        
    for year in years:
        
        daylist = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        if (year%4==0 and year%100!=0) or (year%400==0) : 
            
            daylist = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        for month in months:
            
            days = np.arange(1,daylist[month-1]+1)

            for day in days:
                
                scale = np.mean(data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['scale'])
                scalemean.append(scale)
            
    return scalemean

In [16]:
scalemean = scale_mean_interval(1999,2021,1,13)

In [17]:
# 區間最大規模

def scale_max_interval(startyear,endyear,startmonth,endmonth):
    
    scalemax = []
    
    years = np.arange(startyear,endyear)
    months = np.arange(startmonth,endmonth)
        
    for year in years:
        
        daylist = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        if (year%4==0 and year%100!=0) or (year%400==0) : 
            
            daylist = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        for month in months:
            
            days = np.arange(1,daylist[month-1]+1)

            for day in days:
            
                scale = np.max(data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['scale'])
                scalemax.append(scale)
            
    return scalemax

In [18]:
scalemax = scale_max_interval(1999,2021,1,13)

In [19]:
# 最大規模差值

def scale_max_difference_interval():
    
    difference = np.array(scalemax)-np.array(GRinterval[3])
            
    return difference

In [20]:
scalemaxdifference = list(scale_max_difference_interval())

In [21]:
# 經度緯度偏向

def lonlat_mean_interval(startyear,endyear,startmonth,endmonth):
    
    longitude,latitude,i,maxindex,maxlen = [],[],0,0,0
    
    years = np.arange(startyear,endyear)
    months = np.arange(startmonth,endmonth)
        
    for year in years:
        
        daylist = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        if (year%4==0 and year%100!=0) or (year%400==0) : 
            
            daylist = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        for month in months:
            
            days = np.arange(1,daylist[month-1]+1)

            for day in days:
                
                maxindex = np.where(data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['scale']==scalemax[i])[0][0]
                
                lon = data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['longitude'][maxindex+maxlen]
                lat = data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['latitude'][maxindex+maxlen]
                                                                                                    
                longitude.append(lon)
                latitude.append(lat)
                
                maxlen = maxlen+len(data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['scale'])
                                                                                                    
                i = i+1
            
    return longitude,latitude

In [22]:
lonlatmax = lonlat_mean_interval(1999,2021,1,13)

In [23]:
# 深度偏向

def depth_mean_interval(startyear,endyear,startmonth,endmonth):
    
    depthmean,i,maxlen = [],0,0
    
    years = np.arange(startyear,endyear)
    months = np.arange(startmonth,endmonth)
        
    for year in years:
        
        daylist = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        if (year%4==0 and year%100!=0) or (year%400==0) : 
            
            daylist = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        for month in months:
            
            days = np.arange(1,daylist[month-1]+1)

            for day in days:
                
                maxindex = np.where(data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['scale']==scalemax[i])[0][0]
                
                depth = data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['depth'][maxindex+maxlen]
                depthmean.append(depth)
                
                maxlen = maxlen+len(data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['scale'])
                                                                                                    
                i = i+1
            
    return depthmean

In [24]:
depthmax = depth_mean_interval(1999,2021,1,13)

In [25]:
# 區間能量累積

def energy_sum_interval(startyear,endyear,startmonth,endmonth):
    
    energysum = []
    
    years = np.arange(startyear,endyear)
    months = np.arange(startmonth,endmonth)
        
    for year in years:
    
        daylist = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        if (year%4==0 and year%100!=0) or (year%400==0) : 
            
            daylist = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        for month in months:
            
            days = np.arange(1,daylist[month-1]+1)

            for day in days:
            
                energy = sum(10**(data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['scale']*1.5+11.8-7))
                energysum.append(energy)
            
    return energysum

In [26]:
energysum = energy_sum_interval(1999,2021,1,13)

In [27]:
# 臨界規模

def Critical_scale_interval(startyear,endyear,startmonth,endmonth):
    
    Criticalscale6,Criticalscale5,Criticalscale4 = [],[],[]
    
    years = np.arange(startyear,endyear)
    months = np.arange(startmonth,endmonth)
        
    for year in years:
        
        daylist = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        if (year%4==0 and year%100!=0) or (year%400==0) : 
            
            daylist = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        for month in months:
            
            days = np.arange(1,daylist[month-1]+1)

            for day in days:
            
                scale = data[(data['year']==year)&(data['month']==month)&(data['day']==day)]['scale']

                if True in np.array(scale>=6):
                    Critical6 = 1
                else:
                    Critical6 = 0

                if True in np.array(scale>=5):
                    Critical5 = 1
                else:
                    Critical5 = 0

                if True in np.array(scale>=4):
                    Critical4 = 1
                else:
                    Critical4 = 0

                Criticalscale6.append(Critical6)
                Criticalscale5.append(Critical5)
                Criticalscale4.append(Critical4)
            
    return Criticalscale4,Criticalscale5,Criticalscale6

In [28]:
Criticalscale = Critical_scale_interval(1999,2021,1,13)

In [60]:
# 資料整合

def Data_integration():
    
    TECdata = pd.DataFrame()
    
    TECdata['Criticalscale4'] = Criticalscale[0]
    TECdata['Criticalscale5'] = Criticalscale[1]
    TECdata['Criticalscale6'] = Criticalscale[2]
    
    TECdata['CriticalscaleM'] = scalemax
    
    TECdata['lon'] = lonlatmax[0]
    TECdata['lat'] = lonlatmax[1]
    TECdata['dapth'] = depthmax
    
    TECdata['GRa'] = GRinterval[0]
    TECdata['GRb'] = GRinterval[1]
    TECdata['GRloss'] = GRinterval[2]
    
    TECdata['scalemean'] = scalemean
    TECdata['scalemax'] = scalemax
    
    TECdata['scalemaxexpected'] = GRinterval[3]
    TECdata['scalemaxdifference'] = scalemaxdifference
    
    TECdata['energysum'] = energysum
    
    TECdata = pd.concat([TECdata,Probabilityday.iloc[:,:55]],axis=1)
    
    return TECdata

In [61]:
dataset = Data_integration()

In [65]:
# 一週整合

def OneWeek():
        
    X,C4,C5,C6,CM,lON,LAT,DAPTH,SM,maxindex,maxlen = [],[],[],[],[],[],[],[],[],0,0

    for i in range(int(len(dataset)/7)):

        X.append(np.mean(dataset.iloc[i*7:(i+1)*7,:]))
        
        C4.append(max(dataset.Criticalscale4.iloc[i*7:(i+1)*7]))
        C5.append(max(dataset.Criticalscale5.iloc[i*7:(i+1)*7]))
        C6.append(max(dataset.Criticalscale6.iloc[i*7:(i+1)*7]))
        CM.append(max(dataset.CriticalscaleM.iloc[i*7:(i+1)*7]))
        
        maxindex = np.where(dataset.iloc[i*7:(i+1)*7].scalemax==max(dataset.iloc[i*7:(i+1)*7].scalemax))[0][0]
        
        lON.append(dataset['lon'][maxindex+maxlen])
        LAT.append(dataset['lat'][maxindex+maxlen])
        DAPTH.append(dataset['dapth'][maxindex+maxlen])
        
        SM.append(max(dataset.scalemax.iloc[i*7:(i+1)*7]))
        
        maxlen = maxlen+7

    PartD = pd.DataFrame(X)

    PartD.Criticalscale4 = C4
    PartD.Criticalscale5 = C5
    PartD.Criticalscale6 = C6
    PartD.CriticalscaleM = CM
    
    PartD.lon = lON
    PartD.lat = LAT
    PartD.dapth = DAPTH
    PartD.scalemax = SM
        
    return PartD

In [66]:
OneWeekData = OneWeek()

In [67]:
# 兩週整合

def TwoWeek():

    X,C4,C5,C6,CM,lON,LAT,DAPTH,SM,maxindex,maxlen = [],[],[],[],[],[],[],[],[],0,0

    for i in range(int(len(dataset)/14)):

        X.append(np.mean(dataset.iloc[i*14:(i+1)*14,:]))
        C4.append(max(dataset.Criticalscale4.iloc[i*14:(i+1)*14]))
        C5.append(max(dataset.Criticalscale5.iloc[i*14:(i+1)*14]))
        C6.append(max(dataset.Criticalscale6.iloc[i*14:(i+1)*14]))
        CM.append(max(dataset.CriticalscaleM.iloc[i*14:(i+1)*14]))
        
        maxindex = np.where(dataset.iloc[i*14:(i+1)*14].scalemax==max(dataset.iloc[i*14:(i+1)*14].scalemax))[0][0]
        
        lON.append(dataset['lon'][maxindex+maxlen])
        LAT.append(dataset['lat'][maxindex+maxlen])
        DAPTH.append(dataset['dapth'][maxindex+maxlen])
        
        SM.append(max(dataset.scalemax.iloc[i*14:(i+1)*14]))
        
        maxlen = maxlen+14

    PartD = pd.DataFrame(X)

    PartD.Criticalscale4 = C4
    PartD.Criticalscale5 = C5
    PartD.Criticalscale6 = C6
    PartD.CriticalscaleM = CM
    
    PartD.lon = lON
    PartD.lat = LAT
    PartD.dapth = DAPTH
    PartD.scalemax = SM
        
    return PartD

In [68]:
TwoWeekData = TwoWeek()

In [69]:
# 三週整合

def ThreeWeek():

    X,C4,C5,C6,CM,lON,LAT,DAPTH,SM,maxindex,maxlen = [],[],[],[],[],[],[],[],[],0,0

    for i in range(int(len(dataset)/21)):

        X.append(np.mean(dataset.iloc[i*21:(i+1)*21,:]))
        C4.append(max(dataset.Criticalscale4.iloc[i*21:(i+1)*21]))
        C5.append(max(dataset.Criticalscale5.iloc[i*21:(i+1)*21]))
        C6.append(max(dataset.Criticalscale6.iloc[i*21:(i+1)*21]))
        CM.append(max(dataset.CriticalscaleM.iloc[i*21:(i+1)*21]))
        
        maxindex = np.where(dataset.iloc[i*21:(i+1)*21].scalemax==max(dataset.iloc[i*21:(i+1)*21].scalemax))[0][0]
        
        lON.append(dataset['lon'][maxindex+maxlen])
        LAT.append(dataset['lat'][maxindex+maxlen])
        DAPTH.append(dataset['dapth'][maxindex+maxlen])
        
        SM.append(max(dataset.scalemax.iloc[i*21:(i+1)*21]))
        
        maxlen = maxlen+21

    PartD = pd.DataFrame(X)

    PartD.Criticalscale4 = C4
    PartD.Criticalscale5 = C5
    PartD.Criticalscale6 = C6
    PartD.CriticalscaleM = CM
    
    PartD.lon = lON
    PartD.lat = LAT
    PartD.dapth = DAPTH
    PartD.scalemax = SM
        
    return PartD

In [70]:
ThreeWeekData = ThreeWeek()

In [71]:
# 四週整合

def FourWeek():

    X,C4,C5,C6,CM,lON,LAT,DAPTH,SM,maxindex,maxlen = [],[],[],[],[],[],[],[],[],0,0

    for i in range(int(len(dataset)/28)):

        X.append(np.mean(dataset.iloc[i*28:(i+1)*28,:]))
        C4.append(max(dataset.Criticalscale4.iloc[i*28:(i+1)*28]))
        C5.append(max(dataset.Criticalscale5.iloc[i*28:(i+1)*28]))
        C6.append(max(dataset.Criticalscale6.iloc[i*28:(i+1)*28]))
        CM.append(max(dataset.CriticalscaleM.iloc[i*28:(i+1)*28]))
        
        maxindex = np.where(dataset.iloc[i*28:(i+1)*28].scalemax==max(dataset.iloc[i*28:(i+1)*28].scalemax))[0][0]
        
        lON.append(dataset['lon'][maxindex+maxlen])
        LAT.append(dataset['lat'][maxindex+maxlen])
        DAPTH.append(dataset['dapth'][maxindex+maxlen])
        
        SM.append(max(dataset.scalemax.iloc[i*28:(i+1)*28]))
        
        maxlen = maxlen+28

    PartD = pd.DataFrame(X)

    PartD.Criticalscale4 = C4
    PartD.Criticalscale5 = C5
    PartD.Criticalscale6 = C6
    PartD.CriticalscaleM = CM
    
    PartD.lon = lON
    PartD.lat = LAT
    PartD.dapth = DAPTH
    PartD.scalemax = SM
        
    return PartD

In [72]:
FourWeekData = FourWeek()

In [73]:
# 儲存資料

def Store_data():
    
    filePath = 'D:\Dataset\Earthquake\Calculus/1_CWB.xlsx'
    dataset.to_excel(filePath,index=False)
    
    filePath = 'D:\Dataset\Earthquake\Calculus/7_CWB.xlsx'
    OneWeekData.to_excel(filePath,index=False)
    
    filePath = 'D:\Dataset\Earthquake\Calculus/14_CWB.xlsx'
    TwoWeekData.to_excel(filePath,index=False)
    
    filePath = 'D:\Dataset\Earthquake\Calculus/21_CWB.xlsx'
    ThreeWeekData.to_excel(filePath,index=False)

    filePath = 'D:\Dataset\Earthquake\Calculus/28_CWB.xlsx'
    FourWeekData.to_excel(filePath,index=False)


In [74]:
Store_data()